# Searchlight Analysis
Searchlight Analysis is used to sequentially analyze small groups of voxels in the brain in order to identify regions of interest. The technique is designed in the following steps:

1. Define a sphere of voxels around a seed voxel
2. Extract the time series from each voxel in the sphere
3. Concatenate the time series into a feature vector
4. Train a classifier on the feature vector and the labels
5. Use the classifier to predict the labels of the seed voxel
6. Repeat steps 1-5 for all seed voxels
7. Aggregate the results across all seed voxels to form a statistical map


In [3]:
# Load dependencies
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline

In [ ]:
# Set up data

from preprocessing import load_cleaned_data
cleaned_data_audio, cleaned_data_visual = load_cleaned_data()

# Convert 4D fMRI to a 2D array (samples per timepoint x features)
audio_data = cleaned_data_audio.get_fdata()
visual_data = cleaned_data_visual.get_fdata()

# Reshape data: (x, y, z, time) -> (time, x*y*z)
X_audio = audio_data.reshape(audio_data.shape[-1], -1)
X_visual = visual_data.reshape(visual_data.shape[-1], -1)

# Create labels (0 for audio, 1 for visual)
y_audio = np.zeros(X_audio.shape[0])
y_visual = np.ones(X_visual.shape[0])

# combine datasets to create a single feature matrix and labels
X = np.vstack((X_audio, X_visual))
y = np.hstack((y_audio, y_visual))

In [1]:
# Set up Linear SVM classifier and train the new feature matrix


# Set up model pipeline
pipeline = make_pipeline(StandardScaler(), LinearSVC( random_state = 42, tol = 1e-4, C = 1.0, max_iter = 2000))
pipeline = Pipeline(steps = [('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

# set up features
n_voxels = 1000
X, y = make_classification(n_features=n_voxels, random_state=42)

# Train model
pipeline.fit(X, y)

# Test print the model score
print(pipeline.score(X, y))

1.0
